In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
pinecone_api = os.getenv('PINE_CONE_API')

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [4]:
from pinecone import  Pinecone, ServerlessSpec

f:\Gen Ai\aienv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
index_name = "hybrid-search-langchain-pinecone"

pc = Pinecone(api_key= pinecone_api)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension= 384,
        metric='dotproduct',
        spec = ServerlessSpec(cloud='aws', region='us-east-1')
    )

In [6]:
index = pc.Index(index_name)
index

In [7]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [10]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "in 2021, I visited New Orleans"
]

bm25_encoder.fit(sentences)

bm25_encoder.dump('bm25_values.json')

100%|██████████| 3/3 [00:00<00:00, 2919.47it/s]


In [12]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index= index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000210A7BE00D0>, index=<pinecone.data.index.Index object at 0x00000210EA238210>)

In [13]:
retriever.add_texts(
    [
         "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "in 2021, I visited New Orleans"
    ]
)

100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


In [16]:
retriever.invoke("what city did visit in 20222")

[Document(metadata={'score': 0.351737559}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.323118508}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.302886486}, page_content='in 2021, I visited New Orleans')]